# Fase 7 EDR: Signal Feature extraction
 
Al fin y al cabo lo que estamos buscando es eficiencia en los aterrzajes. Podemos analizar partes de la `serie temporal` que es cada experimento, y sacar variables significativas, como cuanta desviación hay en el eje x, velocidad en el momento del aterrizaje, etc.
 
Atributos a implementar:

1. No sean extremadamente largos
2. Tengan poca variabilidad en el eje x (calcular la variación de X sobre la posición inicial que es cero)
3. La posición de Y debe terminar en cero
4. Las 
5. Analizar la posición de la nave respecto a x antes y después del aterrizaje. Cualquier variación tras haber tocado el suelo probablemente implica baja eficiencia. La idea es mantenerse en 0 o alcanzar esa posición antes de tocar el suelo.
6. En el caso de posiciones de x con comportaminetos extraños, quizá podemos reconstruir los cambios de posición usando la velocidad y un ODE. (La velocidad es la derivada de la función de la posición)
7. Utilizar los outputs de `getLandedExperiments.py` para crear una variable que indique si el experimento ha aterrizado correctamente o no.
8. <span style="color:red">Dudoso:</span> Suavidad en las funciones de aceleración

## Cosas que estudiar

Mejorar la smooth function, cuando la Y no cae drásticamente a 0 pero sí a un numero cercano, no se corrige el valor.

### Implementaciones
#### 1. No sean extremadamente largos
P

In [ ]:

def stdXdeviation(exp: pd.DataFrame) -> NDArray:
    var = sum(exp.x_pos**2)/(exp.shape[0]-1)
    return np.sqrt(var)